In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [3]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 00:54:51 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 00:54:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 00:54:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 00:54:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
dbtable = "inventory"

In [6]:
df_inventory = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_inventory.createOrReplaceTempView(dbtable)

In [8]:
dbtable = "car"

In [9]:
df_car = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [10]:
df_car.createOrReplaceTempView(dbtable)

In [11]:
dbtable = "store"

In [12]:
df_store = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [13]:
df_store.createOrReplaceTempView(dbtable)

In [14]:
dbtable = "address"

In [15]:
df_address = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [16]:
df_address.createOrReplaceTempView(dbtable)

In [17]:
dbtable = "city"

In [18]:
df_city = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [19]:
df_city.createOrReplaceTempView(dbtable)

In [20]:
dbtable = "country"

In [21]:
df_country = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [22]:
df_country.createOrReplaceTempView(dbtable)

In [23]:
dim_inventory = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY i.inventory_id) AS inventory_id,
i.inventory_id AS inventory_key,
i.fuel_type AS inventory_fuel_type,
i.production_year AS inventory_production_year,
i.store_id AS inventory_store_key,
c.producer AS inventory_producer,
c.model AS inventory_model,
a.address AS inventory_address,
a.postal_code AS inventory_postal_code,
ci.city AS inventory_city,
co.country AS inventory_country
FROM inventory AS i
JOIN store AS s ON i.store_id = s.store_id
JOIN car AS c ON i.car_id = c.car_id
JOIN address AS a ON s.address_id = a.address_id
JOIN city AS ci ON a.city_id = ci.city_id
JOIN country co ON ci.country_id = co.country_id 
""")

In [24]:
dim_inventory.show()

24/01/09 00:55:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:55:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:55:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:55:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:55:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:55:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0

+------------+-------------+-------------------+-------------------------+-------------------+------------------+---------------+--------------------+---------------------+--------------+-----------------+
|inventory_id|inventory_key|inventory_fuel_type|inventory_production_year|inventory_store_key|inventory_producer|inventory_model|   inventory_address|inventory_postal_code|inventory_city|inventory_country|
+------------+-------------+-------------------+-------------------------+-------------------+------------------+---------------+--------------------+---------------------+--------------+-----------------+
|           1|            1|             Petrol|                     2016|                  5|               Kia|            Rio|         Wolności 90|               42-625|      Katowice|           Poland|
|           2|            2|             Petrol|                     2016|                  5|            Toyota|          Yaris|         Wolności 90|               42-625|    

In [25]:
dim_inventory.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_inventory")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:56:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0